In [1]:
import tweepy
from sqlalchemy import create_engine, Column, Integer, String, Text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from datetime import datetime, timedelta

# Twitter API credentials
 
access_token= '358138140-mwHh89hZAvtpnYknXUFmuntAZJFdFvvEFc0sdv5R'
access_token_secret = '2tFE2l46VqEKbtciXakCxOc12ByX8j8oOfjuj8zeOO5KH'
consumer_key= 'UugW7t0OUYczGoBBHrN33YXAT'
consumer_secret = 'WtbLz0peBbvwk49uUR7kVi5SGnYUEKhNoJ6Njai6lHzCf7qvl5'

# Authenticate with the Twitter API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Connect to the Twitter API
api = tweepy.API(auth)


In [2]:
import tweepy
import datetime
query = 'depress'



In [3]:
import datetime
datetime.timedelta(days=10)

datetime.timedelta(days=10)

In [4]:
# Iterate over each day in the date range
import pymongo

today = datetime.datetime.now().date()
step1 = today - datetime.timedelta(days=365)
for date in (today - datetime.timedelta(n) for n in range(0, (today - step1).days,30)):
    print(date)


2023-05-08
2023-04-08
2023-03-09
2023-02-07
2023-01-08
2022-12-09
2022-11-09
2022-10-10
2022-09-10
2022-08-11
2022-07-12
2022-06-12
2022-05-13


In [4]:
date

datetime.date(2022, 5, 13)

In [5]:
import pandas as pd
df = pd.DataFrame()
def run_step(today,collection):
    # Search for tweets with the query and get 10 tweets per page
    j= 0
    
    since_date = today - datetime.timedelta(days=30)

    # Perform the search
    tweets = tweepy.Cursor(api.search_tweets,
                q=query,
                lang="en",
                since_id=since_date, until=today).items()

    # Print out the tweets
    for tweet in tweets:
        j = j + 1
        if j > 200:
            break
        tweet_op = {
                'Text' : tweet.text,
                'Date': tweet.created_at
            }
        global df
        df = df.append(tweet_op, ignore_index  = True)
        # collection.insert_one(tweet_op)

    print(today,since_date, j)
    return today - datetime.timedelta(days=30)


In [16]:
import tweepy
import pandas as pd
import time
from datetime import datetime, timedelta


# Authenticate with the Twitter API
auth = tweepy.OAuth1UserHandler(
    consumer_key, consumer_secret, access_token, access_token_secret)

api = tweepy.API(auth)

# Set the query term and date range
query = "depression"
start_date = datetime.now() - timedelta(days=365)
end_date = datetime.now()

# Initialize the dataframe to store the tweets
df = pd.DataFrame(columns=['id', 'created_at', 'text', 'user_screen_name'])

# Set the initial batch of tweets to retrieve
tweets = tweepy.Cursor(api.search_tweets, q=query, lang='en', tweet_mode='extended',
                       since_id=start_date.strftime('%Y-%m-%d'), until=end_date.strftime('%Y-%m-%d')).items()

# Retrieve tweets until the desired number of tweets is reached or the end of the date range is reached
tweet_count = 0
while tweet_count < 1000:
    try:
        # Get the next batch of tweets
        tweet = tweets.next()

        # Check if the tweet meets the criteria (at least 200 tweets per week)
        created_at = datetime.strptime(str(tweet.created_at)[:19], '%Y-%m-%d %H:%M:%S')
        week_start = start_date - timedelta(days=start_date.weekday())
        week_end = week_start + timedelta(days=6)
        if week_start <= created_at <= week_end:
            tweet_count += 1
            df = df.append({'id': tweet.id_str, 'created_at': tweet.created_at,
                            'text': tweet.full_text, 'user_screen_name': tweet.user.screen_name},
                           ignore_index=True)

        # Sleep for a short time to avoid overusing the API
        time.sleep(1)

    except tweepy.TweepyException:
        # If there is a Tweepy error, wait for 15 minutes before continuing
        print("Hit rate limit. Waiting for 15 minutes...")
        time.sleep(900)

# Save the dataframe to a CSV file
df.to_csv('depression_tweets.csv', index=False)


Hit rate limit. Waiting for 15 minutes...
Hit rate limit. Waiting for 15 minutes...
Hit rate limit. Waiting for 15 minutes...
Hit rate limit. Waiting for 15 minutes...


In [6]:
# Replace <username>, <password>, <host>, and <port> with your database credentials
import time, datetime
from tqdm import tqdm
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["Twitter"]
collection = db["RAW_DATA"]
today = datetime.datetime.now().date()
for i in tqdm(range(0,365,30)):
    today = run_step(today,collection)
    time.sleep(10)
client.close()

  0%|          | 0/13 [00:00<?, ?it/s]

2023-05-08 2023-04-08 201


  8%|▊         | 1/13 [00:19<03:52, 19.37s/it]

2023-04-08 2023-03-09 0


 15%|█▌        | 2/13 [00:29<02:36, 14.20s/it]

...

 85%|████████▍ | 11/13 [02:04<00:21, 10.58s/it]

2022-06-12 2022-05-13 0


 92%|█████████▏| 12/13 [02:14<00:10, 10.53s/it]

2022-05-13 2022-04-13 0


100%|██████████| 13/13 [02:25<00:00, 11.17s/it]


In [11]:
print(today)

2022-04-11


In [38]:
df = pd.read_csv('Depression.csv')
df.head()

,label,text,Date
0,4,What a day here in dub!! The sun is jus shinei...,2022-06-30 08:52:00.819595
1,0,@officialTila I wish I would get triple good ...,2022-11-12 08:52:00.819595
2,4,"Diversity is crazy, susan boyle is hot",2022-11-04 08:52:00.819595
3,0,is tired and wants Matt to come back home &lt;3,2022-09-20 08:52:00.819595
4,4,Typically! After 4 days of great weather - the...,2022-08-31 08:52:00.819595


In [39]:
df.shape

(10000, 3)

# MongoDB 

In [40]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["Twitter"]
collection = db["RAW_DATA"]
data_dict = df.to_dict(orient="records")
# insert the data into the collection
collection.insert_many(data_dict)

In [41]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('localhost', 27017)
db = client['Twitter']
collection = db['RAW_DATA']

# Read data from collection and calculate size
data = list(collection.find())
size = len(data)

# Print size
print(f"The collection contains {size} documents.")


The collection contains 13009 documents.
